In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data.csv', encoding='latin1', header=None)
y = pd.read_csv('labels.csv', encoding='latin1', header=None)

In [3]:
X = np.squeeze(data.values)
y = np.squeeze(y.values)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, make_scorer
from sklearn import tree, svm, neighbors, model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np

Nao rodei experimentos para sklearn.naive_bayes.ComplementNB - está falhando o import

In [5]:
'''
#from sklearn.naive_bayes import ComplementNB
cnb_clf = ComplementNB()
parameters = {
    "complementnb__alpha": [0, 1, 3, 5],
    "complementnb__norm": [True, False],
    
}
cnb_clf.random_state = seed
models.append(("naive_bayes.ComplementNB", cnb_clf, parameters))
'''

'\n#from sklearn.naive_bayes import ComplementNB\ncnb_clf = ComplementNB()\nparameters = {\n    "complementnb__alpha": [0, 1, 3, 5],\n    "complementnb__norm": [True, False],\n    \n}\ncnb_clf.random_state = seed\nmodels.append(("naive_bayes.ComplementNB", cnb_clf, parameters))\n'

In [6]:
seed = 42
np.random.seed(seed)

In [7]:
models = []

'''
random_forest_clf = RandomForestClassifier()
random_forest_clf.random_state = seed
parameters = {
    "randomforestclassifier__n_estimators":[10, 20, 30],
    "randomforestclassifier__max_depth": [None, 5, 10, 15, 20]
}
models.append(("RandomForestClassifier", random_forest_clf, parameters))



mnb_clf = MultinomialNB()
parameters = {
    "multinomialnb__alpha": [0, 1, 3, 5, 10]
}
mnb_clf.random_state = seed
models.append(("naive_bayes.MultinomialNB", mnb_clf, parameters))



decision_tree_clf = tree.DecisionTreeClassifier()
decision_tree_clf.random_state = seed
parameters = {
    "decisiontreeclassifier__max_depth": [None, 2, 5, 8, 10, 15]
}
models.append(("decision tree", decision_tree_clf, parameters))


logistic_regression_clf = LogisticRegression()
logistic_regression_clf.random_state = seed
parameters = {
    "logisticregression__penalty": ['l1', 'l2'],
    "logisticregression__C": [0.01, 0.1, 1, 5, 10, 15, 20]
}
models.append(("logistic regression", logistic_regression_clf, parameters))
'''

svm_clf = svm.SVC()
svm_clf.random_state = seed
parameters = {'svc__kernel':('linear', 'rbf'), 'svc__C':[0.1, 1, 5, 10, 15, 20]}
models.append(("svm", svm_clf, parameters))



mlp_clf = MLPClassifier()
mlp_clf.random_state = seed

parameters = {'mlpclassifier__solver': ['lbfgs'], 
              'mlpclassifier__activation':['relu'], 
              'mlpclassifier__max_iter': [2000], 
              'mlpclassifier__alpha': 10.0 ** -np.arange(1, 10), 
              'mlpclassifier__tol':[1e-3], 
              'mlpclassifier__hidden_layer_sizes':np.arange(10, 15)}

models.append(("mlp", mlp_clf, parameters))





knn_clf = neighbors.KNeighborsClassifier()
knn_clf.random_state = seed
parameters = {
    #"kneighborsclassifier__n_neighbors": [1, 3, 5, 7, 9],
    "kneighborsclassifier__n_neighbors": [1],
    "kneighborsclassifier__metric":["cosine"]
}
models.append(("knn", knn_clf, parameters))


In [8]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, pos_label=None, average='micro'),
           'recall': make_scorer(recall_score, pos_label=None, average='micro'),
           'f1_score': make_scorer(f1_score, pos_label=None, average='micro')}


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import fit_grid_point
from sklearn.pipeline import make_pipeline


In [ ]:
from sklearn import cross_validation
skf = cross_validation.StratifiedKFold(y, n_folds=10, shuffle=True, random_state=seed)
skf

C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


sklearn.cross_validation.StratifiedKFold(labels=[0 0 0 ... 0 0 0], n_folds=10, shuffle=True, random_state=42)

In [ ]:
results = pd.DataFrame()
for name, model, parameter in models:
    print('modelo: ', name)
    print(parameter)
    
    tfidf=TfidfVectorizer(sublinear_tf=True, use_idf=True)
    pipe_model = make_pipeline(tfidf, model)
    gs = GridSearchCV(estimator=pipe_model, param_grid=parameter, scoring=scoring, refit='accuracy', cv = skf, return_train_score=True, verbose=5, n_jobs=4)
    gs.fit(X, y)
    print("Best Estimator: \n{}\n".format(gs.best_estimator_))
    print("Best Parameters: \n{}\n".format(gs.best_params_))
    print("Best Test Score: \n{}\n".format(gs.best_score_))
    d = gs.cv_results_
    d['metodo'] = name
    df = pd.DataFrame.from_dict(d)
    results = results.append(df, ignore_index=True)
    print(results)
    
    results.to_csv('resultados.csv', sep=';', decimal=',')
    print('_______________________________________________________')

modelo:  svm
{'svc__kernel': ('linear', 'rbf'), 'svc__C': [0.1, 1, 5, 10, 15, 20]}
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 20.6min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 175.5min
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed: 336.6min finished


Best Estimator: 
Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...r',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False))])

Best Parameters: 
{'svc__C': 5, 'svc__kernel': 'linear'}

Best Test Score: 
0.978674332319417

    mean_fit_time  mean_score_time  mean_test_accuracy  mean_test_f1_score  \
0       84.579734        32.612105            0.969298            0.969298   
1      211.171917        79.185756            0.832436            0.832436   
2       70.108810        22.651682            0.978474            0.978474   
3      209.624833        78.945165            0.832436            0.832436   
4       95.214962        21.155684            0.978674           

[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 12.3min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 31.9min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 55.2min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 84.7min
[Parallel(n_jobs=4)]: Done 450 out of 450 | elapsed: 86.0min finished


Best Estimator: 
Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...       solver='lbfgs', tol=0.001, validation_fraction=0.1, verbose=False,
       warm_start=False))])

Best Parameters: 
{'mlpclassifier__activation': 'relu', 'mlpclassifier__alpha': 1e-06, 'mlpclassifier__hidden_layer_sizes': 12, 'mlpclassifier__max_iter': 2000, 'mlpclassifier__solver': 'lbfgs', 'mlpclassifier__tol': 0.001}

Best Test Score: 
0.9763512384263128

    mean_fit_time  mean_score_time  mean_test_accuracy  mean_test_f1_score  \
0       84.579734        32.612105            0.969298            0.969298   
1      211.171917        79.185756            0.832436            0.832436   
2       70.108810        22.651682            0.97847

In [ ]:
results